In [0]:
prompt_id = 6
model_name = '1layer+BERT+Overall'
# '1layer+BERT+Overall', '1layerMoT+BERT+Overall'

In [2]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.120.188.218:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 13480171355676814969),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10210250124302162304),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1421759878150347840),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11002728950320394671),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4660658030456541356),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 2670143210546846019),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 12104199566132661424),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 12062207937838207434),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 144481210

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/AES謝"
os.chdir(path)
# os.listdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import argparse
import nltk
import tensorflow as tf
from keras.models import Model
from keras import layers
from keras.layers import Input, concatenate, Embedding, LSTM, Dense, Lambda
from keras.engine.topology import Layer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

nltk.download('stopwords')
nltk.download('punkt')
# stopwords, punkt, 

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# Mean Over Time
class TemporalMean(Layer):
  def __init__(self, **kwargs):
      self.supports_masking = True
      super(TemporalMean, self).__init__(**kwargs)

  def compute_mask(self, input, input_mask=None):
      # do not pass the mask to the next layers
      return None

  def call(self, x, mask=None):
      if mask is not None:
          # mask (batch, time)
          mask = K.cast(mask, K.floatx())
          # mask (batch, x_dim, time)
          mask = K.repeat(mask, x.shape[-1])
          # mask (batch, time, x_dim)
          mask = tf.transpose(mask, [0,2,1])
          x = x * mask
      return K.sum(x, axis=1) / K.sum(mask, axis=1)

  def compute_output_shape(self, input_shape):
      # remove temporal dimension
      return (input_shape[0], input_shape[2])

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def get_clean_essays(Text):
    clean_essays = []
    for essay in Text:
        clean_essays.append(essay_to_wordlist(essay, remove_stopwords=False))
    return clean_essays

def create_dict(text):
    print("create_dict")
    word_index = {}
    word_index['xieyikuan'] = 0
    for sentence in text:
        for word in sentence:
            if word not in word_index:
                word_index[word] = len(word_index)
    word_index['unk'] = len(word_index)
    return word_index

def essay_to_index(essays, word_index):
    Essays = []
    for essay in essays:
        sentence = []
        for sent in essay:
            words = []
            for word in sent:
                words.append(word_index[word])
            sentence.append(words)
        Essays.append(sentence)
    return Essays

def essay_to_index2(essays, word_index):
    Essays = []
    for essay in essays:
        words = []
        for word in essay:
            words.append(word_index[word])
        Essays.append(words)
    return Essays

def get_embeddings_index():
    embeddings_index = {}
    f = open('glove.6B.50d.txt', 'r', encoding='UTF-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

def embeddings_matrix(embeddings_index, word_index, embedding_dim):
    nb_words = len(word_index)
    embedding_matrix = np.zeros((nb_words + 1, embedding_dim))
    for word, i in word_index.items():
        if i > nb_words:
            continue
        embeddings_vector = embeddings_index.get(word)
        if embeddings_vector is not None:
            embedding_matrix[i] = embeddings_vector
        if embeddings_vector is None:
            embedding_matrix[i] = embeddings_index.get('unk')
    return embedding_matrix

In [0]:
csv_input = pd.read_csv(filepath_or_buffer="data_feature/set"+str(prompt_id)+"_feature(26).tsv", delimiter="\t", engine="python", error_bad_lines=False)

Text = csv_input['essay']
Score = csv_input['domain1_score']
Label = csv_input['domain1_score']

# --------
if prompt_id == 1:
    Score = (Score - 2) / 10      # set_1     范围 2-12
elif prompt_id == 2:
    Score = (Score - 1) / 5       # set_2     范围 1-6
elif prompt_id == 3 or prompt_id == 4:
    Score = Score / 3           # set_3,4   范围 0-3
elif prompt_id == 5 or prompt_id == 6:
    Score = Score / 4           # set_5,6   范围 0-4
elif prompt_id == 7:
    Score = Score / 30          # set_7     范围 0-30
elif prompt_id == 8:
    Score = Score / 60          # set_8     范围 0-60

# get Overall Feature
overall_feature = csv_input[['word_count', 'avg_word_len', 'sentence_count', 'avg_sentence_len', 'lemma_count', 'noun_count', 'adj_count', 'adv_count', 'verb_count', 'cc_count',
                             'syllable_count', 'flesch_reading_ease', 'smog_index', 'flesch_kincaid_grade', 'coleman_liau_index', 'automated_readability_index',
                             'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula',
                             'gunning_fog', 'spelling_errors', 'exc_count', 'que_count', 'comma_count', 'stopwords_num']]
# overall_feature = np.array(np.sqrt(overall_feature))
overall_feature = np.array(overall_feature)
num_feature = 25
where_are_nan = np.isnan(overall_feature)
where_are_inf = np.isinf(overall_feature)
overall_feature[where_are_nan] = 0
overall_feature[where_are_inf] = 0
# standardized
stdsc = StandardScaler()
overall_feature = stdsc.fit_transform(overall_feature)
 
# Get the Second Last Layer of BERT
bert = pd.read_csv(filepath_or_buffer="fine_tuning_out/essay_set_"+str(prompt_id)+"_output.tsv", delimiter="\t", engine="python", error_bad_lines=False)
bert = np.array(bert)
max_seq_length = len(bert[0])

# 做字典
clean_essays = get_clean_essays(Text)
word_index = create_dict(clean_essays)

# 做分句
text = np.array(Text)
# print(text)
a = []
for essay in text:
    a.append(sent_tokenize(essay))
texts = np.array(a)

# 做分词
essays = []
for essay in Text:
  essay_idx = essay_to_wordlist(essay, remove_stopwords=False)
  essays.append(essay_idx)
essays = np.array(essays)

Essays = essay_to_index2(clean_essays, word_index)
Essays = np.array(Essays)

if prompt_id == 1:
    max_essay_length = 600
elif prompt_id == 2:
    max_essay_length = 700
elif prompt_id == 3:
    max_essay_length = 256
elif prompt_id == 4:
    max_essay_length = 200
elif prompt_id == 5:
    max_essay_length = 256
elif prompt_id == 6:
    max_essay_length = 256
elif prompt_id == 7:
    max_essay_length = 350
elif prompt_id == 8:
    max_essay_length = 870

essay_pad = pad_sequences(Essays, maxlen=max_essay_length)

embeddings_index = get_embeddings_index()
embeddings_matrix = embeddings_matrix(embeddings_index, word_index, embedding_dim=50)

# 记录成绩
results = []
score_predict_list = []
sssscore = []
llllabel = []
newresults = []
cv = KFold(n_splits=5, shuffle=True, random_state=10)
count = 0
for train, test in cv.split(essay_pad):
    text_train, text_test = essay_pad[train], essay_pad[test]
    label_train, label_test = Score.iloc[train], Score.iloc[test]
    score_train, score_test = Label.iloc[train], Label.iloc[test]
    overall_train, overall_test = overall_feature[train], overall_feature[test]
    bert_train, bert_test = bert[train], bert[test]

    # print('Build model '+str(count)+'...')
    # count += 1

    # -------
    text_input = Input(shape=(max_essay_length, ), name='text_input')
    overall_feature_input = Input(shape=(num_feature,), name='overall_feature_input')
    bert_input = Input(shape=(max_seq_length,), name='bert_input')

    embedding_layer = Embedding(input_dim=len(word_index) + 1,
                                output_dim=50,
                                weights=[embeddings_matrix],
                                input_length=max_essay_length,
                                trainable=True,
                                mask_zero=True)
    x = embedding_layer(text_input)  
    if model_name == "1layer+BERT+Overall":
      x = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(x)
    elif model_name == "1layerMoT+BERT+Overall":
      x = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(x)
      x = TemporalMean()(x)
    x = layers.Dropout(0.5)(x)
    concat_layer = Lambda(concatenate)([x, overall_feature_input, bert_input])
    output = Dense(1, activation='sigmoid')(concat_layer)

    model = Model(inputs=[text_input, overall_feature_input, bert_input], outputs=output)
    # -------

    model.summary()
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001, decay=1e-6), metrics=['mean_absolute_error'])
    model.fit([text_train, overall_train, bert_train], label_train, batch_size=32, epochs=50, verbose=2)

    score_predict = model.predict([text_test, overall_test, bert_test])
    if prompt_id == 1:
        score_predict = score_predict*10+2
    elif prompt_id == 2:
        score_predict = score_predict*5+1
    elif prompt_id == 3:
        score_predict = score_predict*3
    elif prompt_id == 4:
        score_predict = score_predict*3
    elif prompt_id == 5:
        score_predict = score_predict*4
    elif prompt_id == 6:
        score_predict = score_predict*4
    elif prompt_id == 7:
        score_predict = score_predict*30
    elif prompt_id == 8:
        score_predict = score_predict*60

    score_predict = np.around(score_predict)

    sssscore.append(score_predict)
    llllabel.append(score_test)

    result = cohen_kappa_score(score_test.values, score_predict, weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)
    for i in results:
      if i != 0:
        newresults.append(i)

print("Average Kappa score after a 5-fold cross validation: ", np.around(np.array(newresults).mean(), decimals=5))

sssscore = np.array(sssscore)
res = []
for scores in sssscore:
  for score in scores:
    res.append(score)

ll = np.array(llllabel)
res_label = []
for labels in ll:
  for label in labels:
    res_label.append(label)
res_label = np.array(res_label)

QWK = cohen_kappa_score(res_label, res, weights='quadratic')
print("Kappa score after a 5-fold cross validation: ", QWK)

create_dict









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 256)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 256, 50)      263350      text_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 300)          421200      embedding_1[0][0]                
____________________________________________________________